In [11]:
file = r"E:\Projects\Neural Machine Translation\deu.txt"
f = open(file,mode ='rt',encoding ='utf-8')
text = f.read()
f.close()

In [13]:
x = text.split('\n')
p = [i.split('\t') for i in x]
p = [i[:2] for i in p]
p

[['Go.', 'Geh.'],
 ['Hi.', 'Hallo!'],
 ['Hi.', 'Grüß Gott!'],
 ['Run!', 'Lauf!'],
 ['Run.', 'Lauf!'],
 ['Wow!', 'Potzdonner!'],
 ['Wow!', 'Donnerwetter!'],
 ['Duck!', 'Kopf runter!'],
 ['Fire!', 'Feuer!'],
 ['Help!', 'Hilfe!'],
 ['Help!', 'Zu Hülf!'],
 ['Stay.', 'Bleib!'],
 ['Stop!', 'Stopp!'],
 ['Stop!', 'Anhalten!'],
 ['Wait!', 'Warte!'],
 ['Wait.', 'Warte.'],
 ['Begin.', 'Fang an.'],
 ['Do it.', 'Mache es!'],
 ['Do it.', 'Tue es.'],
 ['Go on.', 'Mach weiter.'],
 ['Hello!', 'Hallo!'],
 ['Hello!', 'Sers!'],
 ['Hurry!', 'Beeil dich!'],
 ['Hurry!', 'Schnell!'],
 ['I hid.', 'Ich versteckte mich.'],
 ['I hid.', 'Ich habe mich versteckt.'],
 ['I ran.', 'Ich rannte.'],
 ['I see.', 'Ich verstehe.'],
 ['I see.', 'Aha.'],
 ['I try.', 'Ich versuche es.'],
 ['I try.', 'Ich probiere es.'],
 ['I won!', 'Ich hab gewonnen!'],
 ['I won!', 'Ich habe gewonnen!'],
 ['I won.', 'Ich habe gewonnen.'],
 ['Oh no!', 'Oh, Nein!'],
 ['Relax.', 'Entspann dich.'],
 ['Shoot!', 'Feuer!'],
 ['Shoot!', 'Schieß!'],
 [

In [14]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

# helper function
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

cleaned = list()

# prepare regex for character filtering
re_print = re.compile('[^%s]' % re.escape(string.printable))
# prepare translation table for removing punctuation
table = str.maketrans('', '', string.punctuation)
for pair in p:
    clean_pair = list()
    for line in pair:
        # normalize unicode characters
        line = normalize('NFD', line).encode('ascii', 'ignore')
        line = line.decode('UTF-8')
        # tokenize on white space
        line = line.split()
        # convert to lower
        line = [word.lower() for word in line]
        # remove punctuation from each token
        line = [word.translate(table) for word in line]
        # remove non-printable chars form each token
        line = [re_print.sub('', w) for w in line]
        # remove tokens with numbers in them
        line = [word for word in line if word.isalpha()]
        # store as string
        clean_pair.append(' '.join(line))
    cleaned.append(clean_pair)

# save clean pairs to file
save_clean_data(cleaned, r"E:\Projects\Neural Machine Translation\english-german.pkl")
# spot check
for i in range(100):
    print('[%s] => [%s]' % (cleaned[i][0], cleaned[i][1]))

Saved: E:\Projects\Neural Machine Translation\english-german.pkl
[go] => [geh]
[hi] => [hallo]
[hi] => [gru gott]
[run] => [lauf]
[run] => [lauf]
[wow] => [potzdonner]
[wow] => [donnerwetter]
[duck] => [kopf runter]
[fire] => [feuer]
[help] => [hilfe]
[help] => [zu hulf]
[stay] => [bleib]
[stop] => [stopp]
[stop] => [anhalten]
[wait] => [warte]
[wait] => [warte]
[begin] => [fang an]
[do it] => [mache es]
[do it] => [tue es]
[go on] => [mach weiter]
[hello] => [hallo]
[hello] => [sers]
[hurry] => [beeil dich]
[hurry] => [schnell]
[i hid] => [ich versteckte mich]
[i hid] => [ich habe mich versteckt]
[i ran] => [ich rannte]
[i see] => [ich verstehe]
[i see] => [aha]
[i try] => [ich versuche es]
[i try] => [ich probiere es]
[i won] => [ich hab gewonnen]
[i won] => [ich habe gewonnen]
[i won] => [ich habe gewonnen]
[oh no] => [oh nein]
[relax] => [entspann dich]
[shoot] => [feuer]
[shoot] => [schie]
[smile] => [lacheln]
[sorry] => [entschuldigung]
[ask me] => [frag mich]
[ask me] => [fragt 

In [15]:
# taking first 10,000 data points for training
cleaned = [i for i in cleaned if len(i)==2]
cleaned = cleaned[:10000]

In [23]:
#Train-Test split
train = cleaned[:int(len(cleaned)*0.8)]
test = cleaned[int(len(cleaned)*0.8):]

In [18]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

shuffle(cleaned)
# Splitting into train-test and Eng-Ger
eng_array = [i[0] for i in cleaned]
ger_array = [i[1] for i in cleaned]

# Train
eng_train = [i[0] for i in cleaned[:int(len(cleaned)*0.8)]]
ger_train = [i[1] for i in cleaned[:int(len(cleaned)*0.8)]]

# Test
eng_test =  [i[0] for i in cleaned[int(len(cleaned)*0.8):]]
ger_test =  [i[1] for i in cleaned[int(len(cleaned)*0.8):]]

In [19]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint

def tokenize(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

#Tokenization
eng_tokens = tokenize(eng_array)
ger_tokens = tokenize(ger_array)
max_eng_length = len(max(eng_array))
max_ger_length = len(max(ger_array))

In [21]:
# encode and pad sequences
def encode_sequences(tokenizer, req_length, text):
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X, maxlen=req_length, padding='post')
    return X
 
# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = []
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

def define_model(source_vocab_size, translated_vocab_size, source_timesteps, translated_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(source_vocab_size, n_units, input_length=source_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(translated_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(translated_vocab_size, activation='softmax')))
    return model

eng_vocab_size = len(eng_tokens.word_index) + 1
ger_vocab_size = len(ger_tokens.word_index) + 1

# prepare training data
train_ger = encode_sequences(ger_tokens, max_ger_length, ger_train)
train_eng = encode_sequences(eng_tokens, max_eng_length, eng_train)
train_eng = encode_output(train_eng, eng_vocab_size)

# prepare validation data
test_ger = encode_sequences(ger_tokens, max_ger_length, ger_test)
test_eng = encode_sequences(eng_tokens, max_eng_length, eng_test)
test_eng = encode_output(test_eng, eng_vocab_size)
 
# define model
model = define_model(ger_vocab_size, eng_vocab_size, max_ger_length, max_eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)
# Checkpoint
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# Fitting
model.fit(train_ger, train_eng, epochs=30, batch_size=64, validation_data=(test_ger, test_eng), callbacks=[checkpoint], verbose=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 256)           904448    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 9, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 9, 256)            525312    
_________________________________________________________________
time_distributed (TimeDistri (None, 9, 2171)           557947    
Total params: 2,513,019
Trainable params: 2,513,019
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.g

C:\Users\HP\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


Epoch 1/30
125/125 - 46s - loss: 2.7732 - val_loss: 1.9544

Epoch 00001: val_loss improved from inf to 1.95438, saving model to model.h5
Epoch 2/30
125/125 - 21s - loss: 1.8540 - val_loss: 1.8106

Epoch 00002: val_loss improved from 1.95438 to 1.81056, saving model to model.h5
Epoch 3/30
125/125 - 21s - loss: 1.7606 - val_loss: 1.7726

Epoch 00003: val_loss improved from 1.81056 to 1.77261, saving model to model.h5
Epoch 4/30
125/125 - 21s - loss: 1.7099 - val_loss: 1.7525

Epoch 00004: val_loss improved from 1.77261 to 1.75253, saving model to model.h5
Epoch 5/30
125/125 - 21s - loss: 1.6704 - val_loss: 1.7365

Epoch 00005: val_loss improved from 1.75253 to 1.73648, saving model to model.h5
Epoch 6/30
125/125 - 21s - loss: 1.6271 - val_loss: 1.6947

Epoch 00006: val_loss improved from 1.73648 to 1.69469, saving model to model.h5
Epoch 7/30
125/125 - 21s - loss: 1.5649 - val_loss: 1.6486

Epoch 00007: val_loss improved from 1.69469 to 1.64858, saving model to model.h5
Epoch 8/30
125/12

In [25]:
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
 
def word_for_id(integer, tokenizer):
    # map an integer to a word
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
 
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    #extracting predicted word labels
    integers = [argmax(vector) for vector in prediction]
    target = []
    #matching labels to words
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)
 
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = [], []
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokens, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()])
        predicted.append(translation.split())
    # calculate BLEU score
    print('BLEU-1 (Individual N-Gram): %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2 (Cummulative N-Gram): %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
 
trainX = encode_sequences(ger_tokens, max_ger_length, ger_train)
testX = encode_sequences(ger_tokens, max_ger_length, ger_test)
model = load_model('model.h5')
# test
print('Training Results')
evaluate_model(model, eng_tokens, trainX, train)
# test 
print('Testing Results')
evaluate_model(model, eng_tokens, testX, test)

train
src=[hilf mir mal], target=[give me a hand], predicted=[give me a kiss]
src=[schalte ihn an], target=[turn it on], predicted=[turn it on]
src=[bitte haltet den mund], target=[please shut up], predicted=[please shut up]
src=[ich gehe nie schwimmen], target=[i never swim], predicted=[i never swim]
src=[da fang], target=[here catch], predicted=[here catch]
src=[ich habe kaviar gegessen], target=[i ate caviar], predicted=[i ate ate]
src=[das ist ja seltsam], target=[its strange], predicted=[its too]
src=[werft eine munze], target=[flip a coin], predicted=[flip a coin]
src=[ich war hier], target=[i was here], predicted=[i was here]
src=[ich lese nicht], target=[i dont read], predicted=[i dont get]
BLEU-1 (Individual N-Gram): 0.804324
BLEU-2 (Cummulative N-Gram): 0.303724
test
src=[doch tom hat es gemacht], target=[tom did do it], predicted=[tom told it]
src=[so leicht gebe ich nicht auf], target=[im no quitter], predicted=[i dont not]
src=[es ist so su], target=[its so sweet], predict